# Obtaining Soilmodel from Earthquake Data by Using Deep Learning Methods

## Data Preparation

Seismic data of 06/02/2023 earthquake occured at Kahraman Maras, Turkey is obtained from the database of Kandilli Rasathanesi. The seismic signals are collected at KMRS station and GAZ Station which can be seen from the below figure. The signals are the velocity measurements of that earthqueke and they are processed to preare them for training a neural network model.

In [ ]:
# from obspy import read, UTCDateTime
# import numpy as np

# # Read data from MSeed files
# gaz = read('./src/envs/data/gaz.mseed')
# kmrs = read('./src/envs/data/kmrs.mseed')

# # gaz.plot(size=(800, 400))
# # kmrs.plot(size=(800, 400))

# # gaz_filtered = gaz.copy()
# gaz.filter('lowpass',freq=10.0, corners=100, zerophase=True)
# kmrs.filter('lowpass',freq=10.0, corners=100, zerophase=True)
# # gaz_filtered.plot();

# # Crop data between specific date and time
# start_time = UTCDateTime("2023-02-06T04:00:00.000")
# end_time = UTCDateTime("2023-02-06T04:16:20.000")

# gaz_editted = gaz.cutout(start_time,end_time)
# kmrs_editted = kmrs.cutout(start_time,end_time)

# start_time = UTCDateTime("2023-02-06T04:18:00.000")
# end_time = UTCDateTime("2023-02-06T04:50:00.000")

# gaz_editted = gaz_editted.cutout(start_time,end_time)
# kmrs_editted = kmrs_editted.cutout(start_time,end_time)

# # gaz_editted.plot(size=(800, 400))
# # kmrs_editted.plot(size=(800, 400))

# # Merge data of different channels on single array
# gaz_HHE_data = []
# gaz_HHN_data = []
# gaz_HHZ_data = []
# index_start, index_end = [0, 4, 5], [4, 5, 9]

# for i in range(index_start[0], index_end[0]):
#     gaz_HHE_data = np.concatenate((gaz_HHE_data,gaz_editted[i].data), axis=None)

# for i in range(index_start[1], index_end[1]):
#     gaz_HHN_data = np.concatenate((gaz_HHN_data,gaz_editted[i].data), axis=None)

# for i in range(index_start[2], index_end[2]):
#     gaz_HHZ_data = np.concatenate((gaz_HHZ_data,gaz_editted[i].data), axis=None)

# gaz_HHE_data = gaz_HHE_data.reshape(-1,1)
# gaz_HHN_data = gaz_HHN_data.reshape(-1,1)
# gaz_HHZ_data = gaz_HHZ_data.reshape(-1,1)

# print('Size of HHE Data of GAZ Station:', gaz_HHE_data.shape)
# print('Size of HHN Data of GAZ Station:', gaz_HHN_data.shape)
# print('Size of HHZ Data of GAZ Station:', gaz_HHZ_data.shape)
# print('-------------------------------------------------------')

# kmrs_HHE_data = kmrs_editted[0].data.reshape(-1,1)
# kmrs_HHN_data = kmrs_editted[1].data.reshape(-1,1)
# kmrs_HHZ_data = kmrs_editted[2].data.reshape(-1,1)

# print('Size of HHE Data of KMRS Station:', kmrs_HHE_data.shape)
# print('Size of HHN Data of KMRS Station:', kmrs_HHN_data.shape)
# print('Size of HHZ Data of KMRS Station:', kmrs_HHZ_data.shape)
# print('-------------------------------------------------------')

# measurement_data = np.hstack((gaz_HHE_data,gaz_HHN_data,gaz_HHZ_data))
# input_data = np.hstack((kmrs_HHE_data,kmrs_HHN_data,kmrs_HHZ_data))


## 1. Import Libraries, Read Data from the MSeed Files and Convert into Numpy Array, Define Parameters and Hyperparameters

In [ ]:
import torch
from torch import nn
from datetime import datetime
from src.nnmodel.annmodel import LSTMmodel
import src.envs.environment as envi
import scripts.utils as utl

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

### Prepare Data ###
# start_time = "2023-02-05T03:00:00.000"
# end_time = "2023-02-13T03:59:59.000"
# folder_name = './src/envs/data/Input/'

# station_name, data = envi.edit_stream(start_time=start_time, end_time=end_time, type_of_data='raw',
#                                       input_folder_name=folder_name, save_stream=True, filter=False, filter_freq=10.0)

station_name, main_stream = envi.edit_stream(type_of_data='merged',
                                             input_folder_name='./src/envs/data/', save_stream=False, filter=False, filter_freq=10.0)
print('-------------------------------------------------------')
print(f'Station names are as follows: {station_name}')
print('-------------------------------------------------------')
measurement_data = envi.stream2array(main_stream.select(station=station_name[0]))
input_data = envi.stream2array(main_stream.select(station=station_name[1]))


print(f"Size of Gaziantep Data (States):{measurement_data.shape}")
print(f"Size of Kahraman Maras Data (Disturbance):{input_data.shape}")
print('-------------------------------------------------------')
###

comp_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {comp_device}")

# Define Hyperparameters
HIDDEN_SIZE_1 = 128
HIDDEN_SIZE_2 = 64
LEARNING_RATE = 1e-3


# Define training parameters
terminate_loss = 1e-5
max_epoch = 5e3 # maximum number of epochs for trainig
print_interval = 10 # Print interval during training
batch_size = 262144 # GPU memory will be at the limits like 11.8 GB/12.0 GB for 2^19, 2^18 = 262144 might be better.

## 2. Generate DataLoader and Define Forward Pass Function

In [ ]:
env = envi.environment(states=measurement_data,
                       disturbance=input_data,
                       device=comp_device,
                       scale=True)

Train_dataloader, num_input, num_output, Test_dataloader = env.gen_dataset(
    batch_size=batch_size, shuffle=False, split_data=True)

def forward_pass(data, model: torch.nn.Module):
    return model(data)

## 3. Define Optimizer, Loss Function and Setup Trainer

In [ ]:
nn_model = LSTMmodel(input_size = num_input,
                    out_size = num_output,
                    hidden_size_1 = HIDDEN_SIZE_1,
                    hidden_size_2 = HIDDEN_SIZE_1,
                    device = comp_device).to(comp_device)

# loss_fn = nn.MSELoss()
# loss_fn = nn.L1Loss(reduction="sum")
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(params=nn_model.parameters(), lr=LEARNING_RATE)

# Setup trainer
trainer = utl.Trainer(
    model= nn_model,
    forward=forward_pass,
    optimizer = optimizer,
    loss_fn = loss_fn,
    print_every = print_interval,
    schedular = False,
    patience = 100,
)

## 4. Train Model

In [ ]:
date_time = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name=date_time+"_BestModel_"+str(num_input)

trainer.train(train_data=Train_dataloader,
            max_epoch=max_epoch,
            save_checkpoints=False,
            model_name=model_name,
            eval_data = Test_dataloader,
            evaluation = True,
            writer = writer)

## 5. Make Predictions (Burada SIKINTI VAR!!)

In [ ]:
import matplotlib.pyplot as plt
train_inputs, train_labels = env._data_reshape()

# nn_model.to('cpu')
# loss_fn = nn.L1Loss()
# optimizer = torch.optim.Adam(params=nn_model.parameters(), lr=LEARNING_RATE)

# train_inputs, train_labels = train_inputs.to('cpu'), train_labels.to('cpu')

with torch.inference_mode():
    predictions = nn_model(train_inputs)
    loss = loss_fn(predictions, train_labels)
    print(f"Total Calculated Loss Value is {loss}")
labels = ["HHE", "HHN", "HHZ"]

for i in range(3):
    plt.plot(predictions[:,i].cpu(), 'g', linewidth=2, label=f'Predictions for {labels[i]}')
    plt.plot(train_labels[:,i].cpu(), 'r', linewidth=1, label=f'True Value for {labels[i]}')
    plt.title(f'Comparison Plot of Prediction and True Value for {labels[i]}')
    plt.legend()
    plt.grid()
    plt.show()